# Run

In [5]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import datasets
from torchvision.transforms import ToTensor
import torch.optim as optim
from torcheval.metrics.functional import multiclass_f1_score
from torchinfo import summary

import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import pandas as pd
import copy
from tqdm import tqdm
import time
import os
from scipy import signal
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import pickle

In [3]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


In [6]:
with open('Udata.pkl', 'rb') as file:
    Udata = pickle.load(file)

# Vamos remover a atividade 'jumping'
actis = ['climbingdown', 'climbingup', 'lying', 'running', 'sitting', 'standing', 'walking']
posis = ['chest', 'forearm', 'head', 'shin', 'thigh', 'upperarm', 'waist']
users = ['proband' + x for x in np.arange(1,16).astype(str)]
# proband2 não tem acc_climbingup_forearm
users.remove('proband2')
# Vamos remover usuários com menos de 21000 amostras
users.remove('proband1')
users.remove('proband4')
users.remove('proband7')
users.remove('proband14')

In [7]:
def picotar(x, J, passo):
    N = x.shape[0]
    Nj = (N-J)//passo + 1
    X = np.zeros((Nj, J))
    for i in range(Nj):
        X[i,:] = x[(i*passo):(i*passo+J)]
    return X

# MLP cintura larga

In [46]:
nu = 0
nd = 2
fatorEscala = 20
aux1 = []
for na in range(7):
    aux = []
    for i in range(3):
        aux.append(picotar(np.array(Udata[nu][nd])[na, :, i]/fatorEscala, 30, 15))
    aux1.append(np.hstack(aux))
X = np.vstack(aux1)
N = X.shape[0]

In [47]:
X_train, X_test, = train_test_split(
    X, test_size=0.2, random_state=1)
X_train = torch.tensor(X_train, dtype=torch.float32, device=device).detach()
X_test = torch.tensor(X_test, dtype=torch.float32, device=device).detach()

In [48]:
class mlpABA(nn.Module):
    def __init__(self, dim1, dim2):
        super().__init__()
        self.encoder = torch.nn.Sequential(

            nn.Flatten(start_dim=1),
            nn.Linear(dim1, dim2),
            # nn.ReLU(),
            # nn.LeakyReLU(),
            nn.Tanh()
        )

        self.decoder = nn.Linear(dim2, dim1)

    def forward(self, x):
        x = self.encoder(x)
        logits = self.decoder(x)
        return logits

In [49]:
dim1 = X.shape[1]
dim2 = 2*dim1
model = mlpABA(dim1, dim2).to(device)
batch_size = 150
summary(model, input_size=(batch_size, dim1))

Layer (type:depth-idx)                   Output Shape              Param #
mlpABA                                   [150, 90]                 --
├─Sequential: 1-1                        [150, 180]                --
│    └─Flatten: 2-1                      [150, 90]                 --
│    └─Linear: 2-2                       [150, 180]                16,380
│    └─Tanh: 2-3                         [150, 180]                --
├─Linear: 1-2                            [150, 90]                 16,290
Total params: 32,670
Trainable params: 32,670
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 4.90
Input size (MB): 0.05
Forward/backward pass size (MB): 0.32
Params size (MB): 0.13
Estimated Total Size (MB): 0.51

In [34]:
loss_fn = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
n_epochs = 100
batch_start = torch.arange(0, len(X_train), batch_size)
trainLoss = []
valLoss = []
for epoch in range(n_epochs):
    model.train()
    with tqdm(batch_start, unit="batch", mininterval=0, disable=False) as bar:
        bar.set_description(f"Epoch {epoch}")
        for start in bar:
            # take a batch
            X_batch = X_train[start:start+batch_size,:]
            # forward pass
            Xh = model(X_batch)
            loss = loss_fn(Xh, X_batch)
            # backward pass
            optimizer.zero_grad()
            loss.backward()
            # update weights
            optimizer.step()
            # print progress
            bar.set_postfix(loss=float(loss))
    # evaluate accuracy at end of each epoch
    model.eval()
    Xh = model(X_test)
    valLoss.append(float(loss_fn(Xh, X_test)))
    Xh = model(X_train)
    trainLoss.append(float(loss_fn(Xh, X_train)))
fig = go.Figure()
fig.add_trace(go.Scatter(y=valLoss, mode="lines", showlegend=True, name='validação'))
fig.add_trace(go.Scatter(y=trainLoss, mode="lines", showlegend=True, name='treino'))
fig.show()

Epoch 99: 100%|██████████| 47/47 [00:00<00:00, 81.79batch/s, loss=3.61e-6] 


In [37]:
Xh = model(X_test)
Xh = Xh.detach().numpy()
i = np.random.randint(Xh.shape[0])
fig = px.line(Xh[i])
fig.add_trace(go.Scatter(y=X_test[i], mode="lines"))
fig.show()

In [33]:
i = np.random.randint(Xh.shape[0])
fig = px.line(Xh[i])
fig.add_trace(go.Scatter(y=X_test[i], mode="lines"))
fig.show()

In [50]:
nu = 0
dim1 = X.shape[1]
dim2 = 2*dim1
batch_size = 150
fatorEscala = 20
loss_fn = nn.MSELoss()
n_epochs = 100
models = []
for nd in range(7):
    print('Iniciando treinamento do modelo '+posis[nd])
    aux1 = []
    for na in range(7):
        aux = []
        for i in range(3):
            aux.append(picotar(np.array(Udata[nu][nd])[na, :, i]/fatorEscala, 30, 15))
        aux1.append(np.hstack(aux))
    X = np.vstack(aux1)
    X_train, X_test, = train_test_split(
        X, test_size=0.2, random_state=1)
    X_train = torch.tensor(X_train, dtype=torch.float32, device=device).detach()
    X_test = torch.tensor(X_test, dtype=torch.float32, device=device).detach()
    model = mlpABA(dim1, dim2).to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    batch_start = torch.arange(0, len(X_train), batch_size)
    trainLoss = []
    valLoss = []
    for epoch in range(n_epochs):
        model.train()
        with tqdm(batch_start, unit="batch", mininterval=0, disable=False) as bar:
            bar.set_description(f"Epoch {epoch}")
            for start in bar:
                # take a batch
                X_batch = X_train[start:start+batch_size,:]
                # forward pass
                Xh = model(X_batch)
                loss = loss_fn(Xh, X_batch)
                # backward pass
                optimizer.zero_grad()
                loss.backward()
                # update weights
                optimizer.step()
                # print progress
                bar.set_postfix(loss=float(loss))
        # evaluate accuracy at end of each epoch
        model.eval()
        Xh = model(X_test)
        valLoss.append(float(loss_fn(Xh, X_test)))
        Xh = model(X_train)
        trainLoss.append(float(loss_fn(Xh, X_train)))
    fig = go.Figure()
    fig.add_trace(go.Scatter(y=valLoss, mode="lines", showlegend=True, name='validação'))
    fig.add_trace(go.Scatter(y=trainLoss, mode="lines", showlegend=True, name='treino'))
    fig.show()
    models.append(model)

Iniciando treinamento do modelo chest


Epoch 99: 100%|██████████| 53/53 [00:00<00:00, 192.41batch/s, loss=3.69e-6]


Iniciando treinamento do modelo forearm


Epoch 99: 100%|██████████| 53/53 [00:00<00:00, 183.46batch/s, loss=1.12e-5]


Iniciando treinamento do modelo head


...

Epoch 99: 100%|██████████| 53/53 [00:00<00:00, 128.78batch/s, loss=8.32e-6]


Iniciando treinamento do modelo upperarm


Epoch 99: 100%|██████████| 53/53 [00:00<00:00, 130.78batch/s, loss=1.39e-5]


Iniciando treinamento do modelo waist


Epoch 99: 100%|██████████| 53/53 [00:00<00:00, 142.01batch/s, loss=3.53e-6]


In [55]:
lossM = np.zeros((7,7))
for nd in tqdm(range(7)):
    for j in range(7):
        aux1 = []
        for na in range(7):
            aux = []
            for i in range(3):
                aux.append(picotar(np.array(Udata[nu][nd])[na, :, i]/fatorEscala, 30, 15))
            aux1.append(np.hstack(aux))
        X = np.vstack(aux1)
        X = torch.tensor(X, dtype=torch.float32, device=device).detach()
        Xh = models[j](X)
        lossM[nd,j] = loss_fn(Xh, X)

100%|██████████| 7/7 [00:03<00:00,  2.23it/s]


In [58]:
px.imshow(np.log10(lossM),
    labels=dict(x="Dados", y="Modelo", color="MSE"),
        y=posis,
        x=posis)